In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
ERROR: pip's d

In [3]:
# Load the dataset
import pickle

# Specify the path to your pickle file
train_file_path = 'hf_dataset_eu_ai_act_train_dataset.pkl'
val_file_path = 'hf_dataset_eu_ai_act_val_dataset.pkl'

# Open the pickle file in read-binary mode and load the data
with open(train_file_path, 'rb') as file:
    train_loaded_data = pickle.load(file)
    print('Train file loaded successfully')

with open(val_file_path, 'rb') as file:
    val_loaded_data = pickle.load(file)
    print('Validation file loaded successfully')

Train file loaded successfully
Validation file loaded successfully


In [5]:
from datasets import load_dataset, Dataset, DatasetDict
train_dataset = Dataset.from_dict({
    'Instruction': train_loaded_data['Instruction'],
    'Response': train_loaded_data['Response']
})

val_dataset = Dataset.from_dict({
    'Instruction': val_loaded_data['Instruction'],
    'Response': val_loaded_data['Response']
})

datasets = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
os.environ["WANDB_DISABLED"] = "true"
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Instruction', 'Response'],
        num_rows: 707
    })
    validation: Dataset({
        features: ['Instruction', 'Response'],
        num_rows: 79
    })
})

In [9]:
train_dataset = datasets['train']

In [10]:
train_dataset

Dataset({
    features: ['Instruction', 'Response'],
    num_rows: 707
})

In [21]:
eval_dataset = datasets['validation']

In [22]:
eval_dataset

Dataset({
    features: ['Instruction', 'Response'],
    num_rows: 79
})

In [12]:
def tokenize_function(samples):
    # Tokenize the Instruction and Response fields
    instructions_tokens = tokenizer(samples["Instruction"], padding="max_length", truncation=True, max_length=512)
    response_tokens = tokenizer(samples["Response"], padding="max_length", truncation=True, max_length=512)

    # Combine the tokenized fields into one dictionary
    return {
        'input_ids_instruction': instructions_tokens['input_ids'],
        'attention_mask_instruction': instructions_tokens['attention_mask'],
        'input_ids_response': response_tokens['input_ids'],
        'attention_mask_response': response_tokens['attention_mask']
    }

In [23]:
tokenized_dataset_train = train_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/707 [00:00<?, ? examples/s]

In [24]:
tokenized_dataset_eval =  eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [27]:
!pip install scikit-learn


In [28]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


In [43]:
import numpy as np
def compute_metrics(pred):
    logits, labels = pred
    if isinstance(logits, torch.Tensor):
        predictions = logits.argmax(dim=-1).cpu().numpy()
        print("predictions", predictions)
    else:
        predictions = np.argmax(logits, axis=-1)
        print("predictions", predictions)
    labels = labels.cpu().numpy() if isinstance(labels, torch.Tensor) else labels

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [36]:
tokenized_dataset_eval

Dataset({
    features: ['Instruction', 'Response', 'input_ids_instruction', 'attention_mask_instruction', 'input_ids_response', 'attention_mask_response'],
    num_rows: 79
})

In [37]:
tokenized_dataset

Dataset({
    features: ['Instruction', 'Response', 'input_ids_instruction', 'attention_mask_instruction', 'input_ids_response', 'attention_mask_response'],
    num_rows: 707
})

In [38]:
def formatting_func(example):
    text = f"Instruction: {example['Instruction'][0]}\nResponse: {example['Response'][0]}"
    return [text]

In [39]:
data = datasets.map(lambda samples: tokenizer(samples["Instruction"]), batched=True)


Map:   0%|          | 0/707 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [40]:
data["train"]

Dataset({
    features: ['Instruction', 'Response', 'input_ids', 'attention_mask'],
    num_rows: 707
})

In [46]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset_train,
    eval_dataset = tokenized_dataset_eval,
    #eval_dataset=val_loaded_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=int(0.01 * 80),
        max_steps=80,
        # Copied from other hugging face tuning blog posts
        learning_rate=2e-5,
        fp16=True,
        # It makes training faster
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        weight_decay=0.001,


        #Added other than HF Codes
        evaluation_strategy="steps",  # Evaluate during training
        eval_steps=10,
        gradient_checkpointing=True


    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    packing=False,
    # compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/707 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [47]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,2.312500,2.143763
20,2.239400,2.140010
30,2.162800,2.135479
40,2.088900,2.131176
50,2.021500,2.127936
60,1.968500,2.125211
70,1.932400,2.122907
80,1.918700,2.122727


TrainOutput(global_step=80, training_loss=2.105679725110531, metrics={'train_runtime': 84.2385, 'train_samples_per_second': 1.899, 'train_steps_per_second': 0.95, 'total_flos': 390052616601600.0, 'train_loss': 2.105679725110531, 'epoch': 80.0})